In [1]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np

In [2]:
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/cloudflare-status' # one directory
root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/cloudflare-status/2019/20196' # subset of above
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/cloudflare-status*' # all directories

In [3]:
def extract_html_data(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    result = []
    
    regions = ['NA','SA','EU','SP']
    for num, status in enumerate(soup.find_all('td', {'colspan': '3', 'class':'bb top pad8'})):
        try:
            result.append([regions[num], ' '.join(status.text.split())])
        except:
            result.append([regions[num], 'Unknown'])
        
    return [result]

In [4]:
def read_file(file):
    source = file.split('/')[4]
    archive = ZipFile(file, 'r')
    namelist = archive.namelist()
    ret = []
    for item in namelist:
        if 'html' in item:
            html = archive.read(item)
            service = item.split('/')[1].split('.')[0]
            ts = item.split('/')[0]
            data = extract_html_data(html)
            data.insert(0, ts)
            data.insert(0, service)
            data.insert(0, source)
            ret.append(data)
    archive.close()
    return(ret)

In [5]:
files = glob.glob(root_dir + '/**/*.zip', recursive=True)
pool = mp.Pool(mp.cpu_count())
start_time = time.time()
results = pool.map(read_file, [file for file in files])
pool.close()
end_time = time.time()

In [6]:
print(end_time - start_time)

120.6656105518341


In [10]:
res = []
for outer in results:
    for inner in outer:
        res.append(inner)
print(len(res))

926


In [12]:
column_names = ['source','service','timestamp','status']
df = pd.DataFrame(res, columns=column_names)
df.head(50)

0      [[NA, No recent events.], [SA, No recent event...
1      [[NA, No recent events.], [SA, No recent event...
2      [[NA, No recent events.], [SA, No recent event...
3      [[NA, No recent events.], [SA, No recent event...
4      [[NA, No recent events.], [SA, No recent event...
                             ...                        
921    [[NA, No recent events.], [SA, No recent event...
922    [[NA, No recent events.], [SA, No recent event...
923    [[NA, No recent events.], [SA, No recent event...
924    [[NA, No recent events.], [SA, No recent event...
925    [[NA, No recent events.], [SA, No recent event...
Name: status, Length: 926, dtype: object

In [9]:
df.to_csv(r'/home/shane/Documents/thesis/output/cloud-amazon-web-services.csv', index=False)